# Importing libraries

In [ ]:
import os, glob, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd

# Configure directories

In [ ]:
userhome = os.path.expanduser('~')
txt_file = open(userhome + r"/DifferentDiffAlgorithms/SZZ/code_document/project_identity.txt", "r")
pid = txt_file.read().split('\n')
project = pid[0]
bugidentifier = pid[1]
repository = userhome + r'/DifferentDiffAlgorithms/SZZ/datasource/' + project + '/'
analyze_dir = userhome + r'/DifferentDiffAlgorithms/SZZ/projects_analyses/' + project + '/'

print ("Project name = %s" % project)
print ("Project key = %s" % bugidentifier)

# Defining git command function

In [ ]:
def execute_shell_command(buglink, id1, id2, cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return buglink, id1, id2, out, error
    pipe.wait()

In [ ]:
def execute_command(cmd, work_dir):
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Loading file of commit id - parent id

In [ ]:
header = ['bug_id','bugfix_commitID','parent_id']
parent_id = pd.read_csv(analyze_dir + '01_bug_ids_extraction/parentid_of_bugfixcommit.csv')
parent_id = parent_id[header]
parent_id

# Filtering only modified files in bug-fix commit candidates

In [ ]:
files = []
for cmt in range(0,len(parent_id)):
    sys.stdout.write('\rExtracting modified files in commit, row no.: %i' %(cmt+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    
    data_commit = parent_id.iloc[cmt]
    
    diff_cmd = "git diff -w --ignore-blank-lines " + data_commit[2] + " " + data_commit[1] + " --name-status"
    tmp = str(execute_command(diff_cmd, repository))
    tmp = tmp.replace("\\t",",").replace("\\n",",").replace("(b'', b'')","").replace(",', b'')","").replace("(b'","")
    if tmp != '':
        tmp = tmp.split(",")
        if len(tmp) > 2:
            length = len(tmp)/2
        else:
            length = len(tmp)-1
        ii = 0
        for tt in range(0,int(length)):
            if tmp[ii] == 'M':
                tmp2 = [parent_id.iloc[cmt][0], parent_id.iloc[cmt][1], parent_id.iloc[cmt][2], tmp[ii], tmp[ii+1]]
                files.append(tmp2)
            ii += 2

In [ ]:
with open(analyze_dir + "02_diff_extraction/01_modified_files/allmodified_files.csv", 'w') as parent:
    header = ['bug_id','bugfix_commitID','parent_id','status','filename']
    writers = csv.writer(parent, delimiter=',')
    writers.writerow(header)
    for item in files:
        writers.writerow(item)
        
df_file = pd.read_csv(analyze_dir + "02_diff_extraction/01_modified_files/allmodified_files.csv")
df_file = df_file[header]
df_file